## データセットの結合：mergeとjoin
### 関係代数
pandasはpd.merge()関数と、DataFrameのjoin()メソッドで、SeriesおよびDataFrameへの基本的な部品を提供している。<br>
これらは異なるデータを効果的に関係づける。

### 結合の種類
pd.merge()関数は１対１、多対１、多対多の結合を実装する。

#### １対１結合

In [1]:
import numpy as np
import pandas as pd

In [8]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                                   'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                                   'hire_date': [2004, 2008, 2012, 2014]})
print(df1); print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


In [9]:
# pd.merge()でまとめる
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


#### 多対１結合
２つのキー列の１つに重複する項目が含まれている場合の結合

In [10]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                                   'supervisor': ['Carly', 'Guido', 'Steve']})
print(df3); print(df4); print(pd.merge(df3, df4))

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve


#### 多対多結合
両配列のキー列に重複がある場合

In [11]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting', 
                                                      'Engineering', 'Engineering', 'HR', 'HR'],
                                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                                                  'spredsheets', 'organization']})
print(df1); print(df5); print(pd.merge(df1, df5))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR   spredsheets
5           HR  organization
  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR   spredsheets
7      Sue           HR  organization


### キーの指定
#### onキーワード
列名または列名のリストを与えて明示的にキーを指定する。<br>
このオプションは結合する左右のDataFrameに指定された列名が存在するときのみ有効。

In [12]:
print(df1); print(df2); print(pd.merge(df1, df2, on='employee'))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


#### left_onキーワードとright_onキーワード

In [13]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                                       'salary': [70000, 80000, 120000, 90000]})
print(df1); print(df3); print(pd.merge(df1, df3, left_on='employee', right_on='name'))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000


#### left_indexキーワードとright_indexキーワード
列の結合ではなくインデクスを結合する場合に使用する。

In [14]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
print(df1a); print(df2a) 

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014


In [15]:
print(df1a); print(df2a) 
print(pd.merge(df1a, df2a, left_index=True, right_index=True))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


In [16]:
# インデクスをキーとした結合を実行するjoin()メソッドがある
print(df1a); print(df2a) 
print(df1a.join(df2a))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


In [18]:
# left_indexとright_onまたはleft_onとright_indexを組み合われば、インデクスと列の混在した指定が可能
print(df1a);print(df3)
print(pd.merge(df1a, df3, left_index=True, right_on='name'))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000


### 結合に対する集合演算

In [19]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                                       'food': ['fish', 'beans', 'bread']},
                                      columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                                       'drink': ['wine', 'beer']})
print(df6); print(df7); print(pd.merge(df6, df7))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
  drink    name
0  wine    Mary
1  beer  Joseph
   name   food drink
0  Mary  bread  wine


In [20]:
# デフォルトは内部結合（inner）
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


In [21]:
# そのほかにはouter,left,rightが指定できる。
# 外部結合（outer join）は入力値の和集合を返し、全ての欠損値をNAで埋める
print(df6); print(df7); print(pd.merge(df6, df7, how='outer'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
  drink    name
0  wine    Mary
1  beer  Joseph
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer


In [22]:
# 左結合（left join）と右結合（right join）はそれぞれ、左および右のデータセットに存在する行を全て使って結合を行う。
print(df6); print(df7); print(pd.merge(df6, df7, how='left'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
  drink    name
0  wine    Mary
1  beer  Joseph
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine


In [23]:
print(df6); print(df7); print(pd.merge(df6, df7, how='right'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
  drink    name
0  wine    Mary
1  beer  Joseph
     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer


### 列名の重複：sufixesキーワード
結合する２つのDataFrameに競合する列名がある場合

In [25]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                                      'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                                      'rank': [3, 1, 4, 2]})
print(df8); print(df9); print(pd.merge(df8, df9, on='name'))

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


In [26]:
print(df8); print(df9); 
print(pd.merge(df8, df9, on='name', suffixes=["_L", "_R"]))

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2
